In [1]:
import pandas as pd
from datetime import datetime, timedelta

In [2]:
trumptweets = pd.read_csv('./tweets/Trump_Tweets.csv')
trumptweets = trumptweets.dropna()

In [3]:
trumptweets.head(5)

,source,text,created_at,retweet_count,favorite_count,is_retweet,id_str
0,Twitter Media Studio,Thank you to @MarthaRaddatz and @TerryMoran fo...,10-27-2019 21:24:55,11176.0,41087,false,1.188567e+18
1,Twitter for iPhone,RT @StateDept: Last night the United States br...,10-27-2019 16:50:08,16384.0,0,true,1.188498e+18
2,Twitter for iPhone,RT @WhiteHouse: Thank you to the service membe...,10-27-2019 16:49:45,11357.0,0,true,1.188498e+18
3,Twitter for iPhone,https://t.co/7esnNSoa5D,10-27-2019 16:25:12,25546.0,108756,false,1.188492e+18
4,Twitter for iPhone,https://t.co/yJ0VKdNxHP,10-27-2019 14:31:33,22275.0,76549,false,1.188463e+18


In [4]:
trumptweets = trumptweets.drop(["source", "id_str"], axis=1)

In [5]:
trumptweets['created_at']= pd.to_datetime(trumptweets['created_at'], format="%m-%d-%Y %H:%M:%S") 

In [6]:
trumptweets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39096 entries, 0 to 39160
Data columns (total 5 columns):
text              39096 non-null object
created_at        39096 non-null datetime64[ns]
retweet_count     39096 non-null float64
favorite_count    39096 non-null object
is_retweet        39096 non-null object
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 1.8+ MB


In [7]:
trumptweets.head(10)

,text,created_at,retweet_count,favorite_count,is_retweet
0,Thank you to @MarthaRaddatz and @TerryMoran fo...,2019-10-27 21:24:55,11176.0,41087,false
1,RT @StateDept: Last night the United States br...,2019-10-27 16:50:08,16384.0,0,true
2,RT @WhiteHouse: Thank you to the service membe...,2019-10-27 16:49:45,11357.0,0,true
3,https://t.co/7esnNSoa5D,2019-10-27 16:25:12,25546.0,108756,false
4,https://t.co/yJ0VKdNxHP,2019-10-27 14:31:33,22275.0,76549,false
5,As Diwali commences @FLOTUS Melania and I wish...,2019-10-27 10:21:46,23222.0,105322,false
6,Something very big has just happened!,2019-10-27 01:23:39,72300.0,333075,false
7,Governor @MattBevin has done a wonderful job f...,2019-10-26 20:26:34,14671.0,59969,false
8,....Matt has my Complete and Total Endorsement...,2019-10-26 20:26:34,11940.0,48715,false
9,MISSISSIPPI! There is a VERY important electio...,2019-10-26 20:20:04,18282.0,64904,false


In [8]:
# Now we need to get the tweets matched up with the stock movements...

In [9]:
stocks = pd.read_csv('./stocks/spx_preprocessed.csv')
stocks['Date']= pd.to_datetime(stocks['Date']) 

In [10]:
tweet_dates = trumptweets["created_at"].tolist()

In [11]:
# Current stock prices (price before the tweet)
stocks["Date"].max()

Timestamp('2019-11-04 00:00:00')

In [12]:
trumptweets = trumptweets[trumptweets["created_at"] < str(stocks["Date"].max() + timedelta(days=1))]

In [13]:
trumptweets["created_at"].max()

Timestamp('2019-10-27 21:24:55')

In [14]:
# UGLY...
# Need to fix this so it takes time into account - if its before trading hours, we need the day before...
tweet_dates, stock_dates = list(reversed(trumptweets["created_at"].tolist())), stocks["Date"].tolist()
tweet_date_idx, stock_date_idx = len(tweet_dates)-1, len(stock_dates) - 1
before_dates = []
while tweet_date_idx >= 0:
    while stock_date_idx >= 0:
        # If tweet date hour is before 8am, use previous day's close as price (save one as before_date, other as before_metric)
        # If tweet date hour is after 8am, use current day's open as price (if possible)
        if (stock_dates[stock_date_idx]+timedelta(hours=17)) <= tweet_dates[tweet_date_idx]:
            before_dates.append(stock_dates[stock_date_idx])
            tweet_date_idx -=1
        else:
            stock_date_idx -= 1
    if stock_date_idx < 0:
        before_dates.append(None)
    tweet_date_idx -=1

In [15]:
trumptweets["before_date"] = pd.Series(before_dates)

In [16]:
trumptweets["created_at"].min()

Timestamp('2009-05-04 18:54:25')

In [17]:
stocks["Date"].min()

Timestamp('2010-01-04 00:00:00')

In [18]:
trumptweets = trumptweets[trumptweets["created_at"] > str(stocks["Date"].min() - timedelta(days=1))]

In [19]:
# Same deal here...
tweet_dates, stock_dates = list(reversed(trumptweets["created_at"].tolist())), stocks["Date"].tolist()
tweet_date_idx, stock_date_idx = 0, 0
after_dates = []
while tweet_date_idx < len(tweet_dates):
    while stock_date_idx < len(stock_dates) and tweet_date_idx < len(tweet_dates):
        if (stock_dates[stock_date_idx]+timedelta(hours=8)) or >= tweet_dates[tweet_date_idx]:
            after_dates.append(stock_dates[stock_date_idx])
            tweet_date_idx += 1
        else:
            stock_date_idx += 1
    if stock_date_idx >= len(stock_dates):
        after_dates.append(None)
    tweet_date_idx += 1

In [20]:
trumptweets["after_date"] = pd.Series(list(reversed(after_dates)))

In [22]:
trumptweets.head(60)

,text,created_at,retweet_count,favorite_count,is_retweet,before_date,after_date
0,Thank you to @MarthaRaddatz and @TerryMoran fo...,2019-10-27 21:24:55,11176.0,41087,false,2019-10-25,2019-10-28
1,RT @StateDept: Last night the United States br...,2019-10-27 16:50:08,16384.0,0,true,2019-10-25,2019-10-28
2,RT @WhiteHouse: Thank you to the service membe...,2019-10-27 16:49:45,11357.0,0,true,2019-10-25,2019-10-28
3,https://t.co/7esnNSoa5D,2019-10-27 16:25:12,25546.0,108756,false,2019-10-25,2019-10-28
4,https://t.co/yJ0VKdNxHP,2019-10-27 14:31:33,22275.0,76549,false,2019-10-25,2019-10-28
5,As Diwali commences @FLOTUS Melania and I wish...,2019-10-27 10:21:46,23222.0,105322,false,2019-10-25,2019-10-28
6,Something very big has just happened!,2019-10-27 01:23:39,72300.0,333075,false,2019-10-25,2019-10-28
7,Governor @MattBevin has done a wonderful job f...,2019-10-26 20:26:34,14671.0,59969,false,2019-10-25,2019-10-28
8,....Matt has my Complete and Total Endorsement...,2019-10-26 20:26:34,11940.0,48715,false,2019-10-25,2019-10-28
9,MISSISSIPPI! There is a VERY important electio...,2019-10-26 20:20:04,18282.0,64904,false,2019-10-25,2019-10-28
